This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

# Load the results + get generated suffixes

In [9]:
dir_suffix_search = '/home/chansingh/mntv1/suffix_math_long_match'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search,   )
r = r_suffix_search
r.to_pickle('../results/prompt_gen/results_suffix_all.pkl')

100%|██████████| 27/27 [00:03<00:00,  7.16it/s]


In [11]:
r = pd.read_pickle('../results/prompt_gen/results_suffix_all.pkl')

# Look at top suffix candidates

In [13]:
r[r.max_num_tokens == 6]

,task_name,n_shots,max_dset_size,max_digit,template_num_init_string,template_num_task_phrasing,train_split_frac,checkpoint,max_num_tokens,beam_size,...,Recall @ 25 suffixes,Recall @ 30 suffixes,Recall @ 40 suffixes,Recall @ 50 suffixes,Recall @ 75 suffixes,Recall @ 100 suffixes,Recall @ 150 suffixes,Recall @ 200 suffixes,reciprocal_rank,reciprocal_rank_multi
0,task1146_country_capital,1,10000,10,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,False,False,False,False,False,True,True,True,0.010417,0.016129
1,task1147_country_currency,1,10000,10,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,False,False,False,True,True,True,True,True,0.020833,0.020408
5,task1149_item_check_edible,1,10000,10,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,False,False,False,False,False,False,False,False,0.000173,NaN
18,task183_rhyme_generation,1,10000,10,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,False,False,False,False,False,False,False,False,0.000155,NaN
21,task1509_evalution_antonyms,1,10000,10,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,False,False,False,False,False,False,False,False,0.001087,0.009174
22,task092_check_prime_classification,1,10000,10,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,False,False,False,False,False,False,False,False,0.000486,NaN
26,add_two,1,5000,100,0,0,0.75,EleutherAI/gpt-j-6B,6,5,...,True,True,True,True,True,True,True,True,0.166667,0.142857


In [18]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1) * (r.checkpoint == 'EleutherAI/gpt-j-6B')]
# r[['suffix_str_added', 'final_answer_added']]
print('shape', r0.shape)
for i in range(r0.shape[0]):
    row = r0.iloc[i]
    print('task', row.task_name)
    candidates, probs = analyze_utils.get_top_candidates_and_probs_suff(row)
    for j in range(5):
        print('\t', candidates[j], probs[j])

shape (27, 52)
task task1146_country_capital


# Assess accuracy of the found suffixes

In [38]:
args = r.iloc[3]

In [39]:
np.random.seed(args.seed)
(dset, dset_test), check_answer_func, descr = data.get_data(
            args, args.task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)

In [43]:
checkpoint = 'EleutherAI/gpt-j-6B' # 'gpt2-medium'
model = prompt_classification.create_model(checkpoint)

In [ ]:
loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix=f'Return the antonym of the input: ', multi_token=True,
)